### Loading Data

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pymongo import MongoClient

file_path = "urban_mobility_dataset.csv"
data = pd.read_csv(file_path)
print("\nNumber of rows and columns:", data.shape)


Number of rows and columns: (1000000, 15)


In [39]:
data.isnull().sum()

timestamp                      0
public_transport_usage         0
traffic_flow                   0
bike_sharing_usage             0
pedestrian_count               0
weather_conditions             0
day_of_week                    0
holiday                        0
event                     799487
temperature                    0
humidity                       0
road_incidents                 0
public_transport_delay         0
bike_availability              0
pedestrian_incidents           0
dtype: int64

In [5]:
pip install pymongo

   ---------------------------------------- 0.0/926.7 kB ? eta -:--:--
   ----------------- --------------------- 419.8/926.7 kB 13.2 MB/s eta 0:00:01
   --------------------------------------  921.6/926.7 kB 14.5 MB/s eta 0:00:01
   ---------------------------------------- 926.7/926.7 kB 9.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   timestamp               1000000 non-null  object 
 1   public_transport_usage  1000000 non-null  int64  
 2   traffic_flow            1000000 non-null  int64  
 3   bike_sharing_usage      1000000 non-null  int64  
 4   pedestrian_count        1000000 non-null  int64  
 5   weather_conditions      1000000 non-null  object 
 6   day_of_week             1000000 non-null  object 
 7   holiday                 1000000 non-null  int64  
 8   event                   200513 non-null   object 
 9   temperature             1000000 non-null  float64
 10  humidity                1000000 non-null  int64  
 11  road_incidents          1000000 non-null  int64  
 12  public_transport_delay  1000000 non-null  float64
 13  bike_availability       1000000 non-null  int64  
 14  ped

Inserting into Mongodb

In [12]:
client = MongoClient("mongodb://localhost:27017/")
db = client["urban_mobility"]
collection = db["raw_data"]

# Insert data into MongoDB
collection.insert_many(data.to_dict(orient="records"))
print("Data inserted into MongoDB collection.")

Data inserted into MongoDB collection.


Data Cleaning

In [14]:
data.isnull().sum()

timestamp                      0
public_transport_usage         0
traffic_flow                   0
bike_sharing_usage             0
pedestrian_count               0
weather_conditions             0
day_of_week                    0
holiday                        0
event                     799487
temperature                    0
humidity                       0
road_incidents                 0
public_transport_delay         0
bike_availability              0
pedestrian_incidents           0
dtype: int64

In [16]:
data.fillna("Unknown", inplace=True)  
data.drop_duplicates(inplace=True)
cleaned_collection = db["cleaned_data"]
cleaned_collection.insert_many(data.to_dict(orient="records"))
print("Cleaned data inserted into MongoDB collection.")

Cleaned data inserted into MongoDB collection.


In [51]:
#After Data cleaing 
print(data.isnull().sum())

timestamp                 0
public_transport_usage    0
traffic_flow              0
bike_sharing_usage        0
pedestrian_count          0
weather_conditions        0
day_of_week               0
holiday                   0
event                     0
temperature               0
humidity                  0
road_incidents            0
public_transport_delay    0
bike_availability         0
pedestrian_incidents      0
dtype: int64


In [18]:
print("\nNumber of rows and columns:", data.shape)


Number of rows and columns: (1000000, 15)


### Aggregated Dataset

In [20]:
print("Column Names in the Dataset:")
print(data.columns)
data.columns = data.columns.str.strip()  # Remove leading/trailing spaces
print("Updated Column Names:")
print(data.columns)
aggregated_data = data.groupby("day_of_week")["public_transport_usage"].sum().reset_index()
print("Aggregated Dataset:")
print(aggregated_data.head())
aggregated_collection = db["aggregated_data"]
aggregated_collection.insert_many(aggregated_data.to_dict(orient="records"))
print("Aggregated data successfully inserted into MongoDB collection.")

Column Names in the Dataset:
Index(['timestamp', 'public_transport_usage', 'traffic_flow',
       'bike_sharing_usage', 'pedestrian_count', 'weather_conditions',
       'day_of_week', 'holiday', 'event', 'temperature', 'humidity',
       'road_incidents', 'public_transport_delay', 'bike_availability',
       'pedestrian_incidents'],
      dtype='object')
Updated Column Names:
Index(['timestamp', 'public_transport_usage', 'traffic_flow',
       'bike_sharing_usage', 'pedestrian_count', 'weather_conditions',
       'day_of_week', 'holiday', 'event', 'temperature', 'humidity',
       'road_incidents', 'public_transport_delay', 'bike_availability',
       'pedestrian_incidents'],
      dtype='object')
Aggregated Dataset:
  day_of_week  public_transport_usage
0      Friday                39132264
1      Monday                39239693
2    Saturday                39215502
3      Sunday                39252867
4    Thursday                39277671
Aggregated data successfully inserted into Mo